# Machine learning II

files needed = ('auto.csv')

This book continues our foray into machine learning. Our goals here are modest. We would like to
1. learn a bit about how machine learning is similar to, and different from, econometrics.  
2. introduce the scikit-learn package which is chock full of 'machine learning' tools. 
3. work on some *validation* methods, which are an important part of the machine learning toolkit. 

In this notebook, we continue to discuss validation methods.

This notebook is loosely based on Chapters 2 and 5 from *An Introduction to Statistical Learning* by James, Witten, Hastie, and Tibshirani. This is an easy to follow introduction that is light on the mathematics behind the methods.

### Cross validation

In the last workbook, we studied *set validation*. 

1. We break our dataset up into two subsets. Call one the *training set* and the other the *testing set*. 
2. We estimate our model on the training data. We measure how well our model fits the data by looking at the mean squared error of the regression or, alternatively, the r-squared, which normalizes the mse by the total squared error. We call these the **training mse or the training $r^2$**. This is usually what we care about when we are doing inference. 
3. Using the X data in the testing data, we use the estimated $f(\;)$ from step 2 to predict value for y
$$\hat{y} = \hat{f}(X_{test}).$$
4. We then compare the predicted ($\hat{y}$) values with the actual ($y$) values in the training data. Again, we can use mse or r-squared to judge how well our predicted data matched the actual data. Call this the **test mse or test $r^2$**. This is what we usually care about when we are interested in prediction. 

In the last notebook we repeatedly performed set validation on a linear model. We found that the reported error measures varied considerably. This is because we were using a different 75% of the observations to estimate the model each time. As we changed the training data set, the estimates changed. 

A related drawback is that we are only using 75% of the observations to estimate the model each time. Typically, the more observations we use, the better our estimates will be, which will lead to better prediction.  

### Leave one out cross-validation (loocv)

Leave one out cross-validation (loocv) addresses the concerns with set validation. The loocv algorithm is 

For each observation in the data $i = 0,...n$
1. The test data is observation $i$
2. The training data is all the data **except** observation $i$
3. Estimate the model on the training data
4. Evaluate the model on the testing data, this generates a mean squared error associated with 'round' $i$

The test mse is the average over the $n$ mses computed:
$$ CV_{(n)} = \frac{1}{n}\sum_{i=1}^nmse_{i}$$

Notice that this is similar in spirit to what we were doing last class. The difference is that we are doing are repeating the estimate exactly $n$ times and that the split into test and training sets is not random. 

Let's try it out. Rather than write code to implement the algorithm above, scikit will do it for us. 

First, load up the auto data again. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# There are a few missing values coded as '?'. 
auto = pd.read_csv('auto.csv', na_values='?')
auto = auto.dropna()
print(auto.info())

We will need the `linear_model` package to run the regression and the `cross_val_score` function from scikit-learn.

In [ ]:
# From scikit-learn import the cross_val_score function
from sklearn.model_selection import cross_val_score

# From scikit-learn, peel off the linear_model package
from sklearn import linear_model 

The `cross_val_score()` function [(docs)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) takes as arguments an `estimator`, in our case ols, the `x data`, the `y data`, a `scoring` criterion, and the number of splits to make to the data, `cv`. 

Two comments:

1. The r-squared is not a good metric of fit (score) in this case, because we are testing on a single observation. The r-squared will be zero always. Instead, we will use the mean squared error. In scikit learn, the **negative** of the mean squared error is returned. This is simply a convention: The package creators want all the scoring criterion to be increasing in fit. (e.g., a larger r-squared is better, a larger (closer to zero) negative mse is better).
2. We set the `cv` parameter to the number of observations in the data. We want to split the data into $n$ chunks (each chunk with one observation) and leave one chunk out each iteration.  

In [ ]:
# estimator = LinearRegression, # X_data = auto[['horsepower', 'weight']], y_data = auto['mpg']

scores = cross_val_score(linear_model.LinearRegression(), auto[['horsepower', 'weight']], auto['mpg'],
                         scoring='neg_mean_squared_error', cv=len(auto['mpg']))

That was easy! The `cross_val_score` function returns an array with the negative mse for each iteration. 

In [ ]:
print('Scores is of type:', type(scores))
print('Length of scores:', len(scores), '\n')
print(scores)

To compute the test mean squared error, 
$$ CV_{(n)} = \frac{1}{n}\sum_{i=1}^nmse_{i}$$
we just compute the mean of scores. I will multiply by -1, so that we have the mse.

In [ ]:
test_mse = scores.mean()
print('The test mse is {0:5.3f}'.format(-test_mse))

### k-fold cross-validation

k-fold cross-validation is a generalization of loocv. An issue with loocv is that we need to estimate the model $n$ times, once for each observation. If there is a lot of data, or if the model is hard to estimate, this approach is may be computationally intensive. 

The solution to this is to break the data up into k sets (or folds) of length n/k, each set containing n/k randomly assigned observations. We then proceed as in loocv, but estimating the model only $i=1,...,k$ times and leaving out set $i$ as the test set. This [figure](http://www-bcf.usc.edu/~gareth/ISL/Chapter5/5.5.pdf) from James, et al. summarizes it nicely. 

In this procedure, the test mse is 
$$ CV_{(k)} = \frac{1}{k}\sum_{i=1}^k mse_{i}$$

Clearly, loocv is just k-fold cv with $k=n$. 

How big is $k$? In practice, people tend to use 5 or 10. 

\[There are additional reasons dealing with bias and variance to use $k<n$, but that discussion is outside the scope of this class.\]

In [ ]:
# estimator = LinearRegression, # X_data = auto[['horsepower', 'weight']], y_data = auto['mpg']

scores_10 = cross_val_score(linear_model.LinearRegression(), auto[['horsepower', 'weight']], auto['mpg'],
                         scoring='neg_mean_squared_error', cv=10)

In [ ]:
print(scores_10)
print('\nThe test mse is {0:5.3f}'.format(-scores_10.mean()))

## Practice

Take a few minutes and try the following. Feel free to chat with those around if you get stuck. The TA and I are here, too.

We will use k-fold cross-validation to compare models with higher-degree polynomials in horsepower.

$$\begin{align}
mpg &= \beta_0 + \beta_1 hp +\epsilon\\
mpg &= \beta_0 + \beta_1 hp + \beta_2 hp^2 + \epsilon\\
mpg &= \beta_0 + \beta_1 hp + \beta_2 hp^2 + \beta_3 hp^3 + \epsilon\\
mpg &= \beta_0 + \beta_1 hp + \beta_2 hp^2 + \beta_3 hp^3 + \beta_4 hp^4 + \epsilon\\
mpg &= \beta_0 + \beta_1 hp + \beta_2 hp^2 + \beta_3 hp^3 + \beta_4 hp^4 + \beta_5 hp^5 + \epsilon
\end{align}$$

1. Create columns named 'hp2' through 'hp5' that correspond to the powers of horsepower. I used a loop.

2. Compute the k-fold validation mse with k=10
$$ CV_{(k)} = \frac{1}{k}\sum_{i=1}^k mse_{i}$$
for each of the models. Save the mse for each model in a list.  

\[I did this by hand (rather than in a loop), which I'm kind of ashamed of. I need to think harder about a way to automate this. If you finish early, try automating this. For sure, we could use patsy to simplify this step...\]

3. Plot the mse (not the negative mse) from each model on the y-axis of a scatter plot, with \[1, 2, 3, 4, 5\] on the x-axis. Which degree of polynomial would you include in your model?   